In [16]:
import os
import cv2
import sys
import pandas as pd
import numpy as np
import copy
from fnmatch import fnmatch 
#from matplotlib import pyplot as plt

In [17]:
def click_images(name):

    print("Take photos for training real-time dataset ")
    print("Press Enter to click photos. Take atleast 10 photos")

    cam = cv2.VideoCapture(0)

    cv2.namedWindow("new_face")

    img_counter =0 

    while True: 
        ret, frame = cam.read()

        cv2.imshow("new_face", frame)

        if not ret : 
            break

        k = cv2.waitKey(1)

        if k%256 == 27:       # ESC for closing 
            print "Closing camera"
            break

        elif k%256 == 13:       # carriage return for clicking photos 

            image_name = "{}_frame_{}.jpg".format(name,img_counter)

            in_path = os.path.realpath("final.ipynb.py")
            folder = "/real_time/%s/"%(name)
            path = os.path.dirname(in_path) + folder

            try:
                os.makedirs(path)
                cv2.imwrite(path+ image_name, frame)
            except:
                cv2.imwrite(path+ image_name, frame)
                


            img_counter+=1
            print "photo %d clicked \n"%(img_counter)

    cam.release()

    cv2.destroyAllWindows()

In [18]:
def findface_haarcascades(name):
    face_cascade = cv2.CascadeClassifier('/home/pranjali/.local/lib/python2.7/site-packages/cv2/data/haarcascade_frontalface_default.xml')

    in_path = os.path.realpath("final.ipynb")
    folder = "/real_time/%s/"%(name)
    path = os.path.dirname(in_path) + folder

    patternmatch="*.jpg"

    Path_to_images=[]
    for dirname, subdirname, files in os.walk(path):
        #for files in subdirname:
            for names in files :
                if fnmatch(names, patternmatch):
                    Path_to_images.append(os.path.join(path,name))
                    image_path = path + names

                    img = cv2.imread(image_path, cv2.COLOR_BGR2GRAY)

                    faces= face_cascade.detectMultiScale(img,1.3,5)

                    for (x,y,w,h) in faces:
                        face_image = img[y:y+h,x:x+w]
                        face_image = cv2.resize(face_image,(32,32))
                        cv2.imwrite(image_path, face_image)


    print "Done extracting faces of %s\n"%(name)

In [19]:
def find_train_images(images_data):

    if images_data == "existing_dataset":
        folder='/faces/'
        patternmatch="*.pgm"

    if images_data== "real_time":
        folder='/real_time/'
        patternmatch="*.jpg"

    in_path = os.path.realpath("PCA.py.ipynb")
    path = os.path.dirname(in_path) + folder

    

    Path_to_images=[]
    for dirname, subdirname, files in os.walk(path):
        #for files in subdirname:
            for name in files :
                if fnmatch(name, patternmatch):
                    Path_to_images.append(os.path.join(dirname,name))

    train_images=[]
    sizes=[]
    for path_of_image in Path_to_images:
        img_2D = cv2.imread(path_of_image, cv2.COLOR_BGR2GRAY)  
        #img_2D = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)	

        img_1D = np.array(img_2D, dtype='float64').flatten() 
        train_images.append(img_1D)
        sizes.append(img_1D.size)

    size_of_image = train_images[0].size

    train_set_size = len(train_images)

    images_1d=np.array(train_images)


    images_1d=images_1d.T


    return images_1d, size_of_image, train_set_size, Path_to_images

In [20]:
def mean1d_(images_1d):
    mean1d=images_1d.mean(axis=1)
    return mean1d

In [21]:
def images_mean_(images_1d,mean1d,train_set_size):
    images_mean=np.zeros([10,112*92],dtype='float64')
    images_mean=copy.deepcopy(images_1d)
    for i in range(0,train_set_size):
        images_mean[:,i]=images_1d[:,i]-mean1d
    return images_mean

In [22]:
def PCA_(images_mean,k):
    CovarianceMatrix=np.matrix(images_mean.transpose()) * np.matrix(images_mean) 
    eigenvalues,eigenvectors = np.linalg.eig(CovarianceMatrix)

    indices_sorted=np.argsort(eigenvalues)[::-1]
    eigenvalues=eigenvalues[indices_sorted]
    eigenvectors=eigenvectors[indices_sorted]

    eigenvalues = eigenvalues[0:k]
    eigenvectors = eigenvectors[0:k,:]

    eigenvectors = eigenvectors.transpose()
    eigenvectors = images_mean * eigenvectors

    norms = np.linalg.norm(eigenvectors,axis=0)
    eigenvectors = eigenvectors/norms

    weights= eigenvectors.transpose()*images_mean

    return eigenvectors, weights

In [23]:
def train(dataset_name):

    # dataset_name : 'existing_dataset' , 'real_time'


    images_1d, size_of_image, train_set_size, Path_to_images=find_train_images(dataset_name)

    mean1d=mean1d_(images_1d)
    images_mean=images_mean_(images_1d,mean1d, train_set_size)

    k = ((images_mean.shape[1])/2)+1

    eigenvectors,weights=PCA_(images_mean,k)


    print "size of images" , size_of_image

    return images_1d, mean1d, eigenvectors, weights, Path_to_images

In [24]:
def test(mean1d, eigenvectors, weights, test_image, Threshold=10000):
    
    test_image = test_image.flatten()
    

    test_image_mean = test_image - mean1d
    
 
    test_image_mean = np.matrix(test_image_mean)
    test_image_mean=test_image_mean.transpose()
    weights_test= eigenvectors.transpose() * test_image_mean
    difference = weights - weights_test
    distance_faces = np.linalg.norm(difference,axis=0)

    
    closest_face_index = np.argmin(distance_faces)
    closest_face_distance = min(distance_faces) 


    if closest_face_distance > Threshold :
        closest_face_distance = -1

    return closest_face_index, closest_face_distance

In [25]:
def test_existing(mean1d, eigenvectors, weights,dataset_name, Threshold=10000):
    if dataset_name == "existing_dataset":
        folder='/test/'
        patternmatch="*.pgm"

    in_path = os.path.realpath("final.ipynb")
    path = os.path.dirname(in_path) + folder


    Path_to_test_images=[]
    for dirname, subdirname, files in os.walk(path):
        for name in files :
            if fnmatch(name, patternmatch):
                Path_to_test_images.append(os.path.join(dirname,name))

    closest_face_index_lst=[]
    closest_face_distance_lst =[]

    for test_image_path in Path_to_test_images:
        img_2D = cv2.imread(test_image_path, cv2.COLOR_BGR2GRAY)  
        closest_face_index, closest_face_distance = test(mean1d, eigenvectors, weights, img_2D)

        closest_face_index_lst.append(closest_face_index)
        closest_face_distance_lst.append(closest_face_distance)

    return Path_to_test_images, closest_face_index_lst, closest_face_distance_lst

In [26]:
def real_time_display(images_1d, mean1d, eigenvectors, weights):
    face_cascade = cv2.CascadeClassifier('/home/pranjali/.local/lib/python2.7/site-packages/cv2/data/haarcascade_frontalface_default.xml')

    cap = cv2.VideoCapture(0)

    while 1:
        ret, img = cap.read()
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        faces= face_cascade.detectMultiScale(gray,1.2,5)

        for (x,y,w,h) in faces:
            face_img=img[y:y+h,x:x+w]
            face_image=face_img[:,:,0]

            face_image=cv2.resize(face_image,(32,32))
            closest_face_index, closest_face_distance = test(images_1d, mean1d, eigenvectors, weights, face_image)
            if closest_face_distance==-1:
                name_of_face_class=''
            else:
                list_of_images = copy.deepcopy(Path_to_images)
                name_of_face_class = os.path.basename(os.path.dirname(list_of_images[closest_face_index]))
                
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
            cv2.putText(img, name_of_face_class, (x,y-2), cv2.FONT_HERSHEY_PLAIN, 2 ,(0,0,255),2)

        cv2.imshow("main",img)
        k = cv2.waitKey(30) & 0xff
        if k == 27 :
            break



    cap.release()
    cap.destroyAllWindows()




In [27]:
def print_help():
    print(""" FACE RECOGNITION 

    Options : 

        1.  Train dataset for real-time and test it 
    

    Please enter an option (1) \n""")

In [28]:
def main():


    print_help()

    command = raw_input()

    if command == '1':
        
        ## train

        # creating real-time dataset
        dataset_name = 'real_time'
        print('Creating real-time dataset\n')
        print("please enter name of class")
        name =raw_input()
        click_images(name)
        findface_haarcascades(name)

        # Training real-time dataset
        print('Training real-time dataset\n')
        images_1d, mean1d, eigenvectors, weights, Path_to_images = train(dataset_name)

        
        print mean1d
        print mean1d.shape
        
        ## test real-time dataset
        print('Testing real-time dataset\n')
        print("starting video camera")
        real_time_display(images_1d, mean1d, eigenvectors, weights)

        print("Done")


    else:
        print_help()
        

In [29]:
if __name__ == '__main__':
    main()

 FACE RECOGNITION 

    Options : 

        1.  Train dataset for real-time and test it 
    

    Please enter an option (1) 

1
Creating real-time dataset

please enter name of class
pj1
Take photos for training real-time dataset 
Press Enter to click photos. Take atleast 10 photos
photo 1 clicked 

photo 2 clicked 

photo 3 clicked 

photo 4 clicked 

photo 5 clicked 

photo 6 clicked 

Closing camera
Done extracting faces of pj1

Training real-time dataset



IndexError: tuple index out of range